# Gopigo navigation & SLAM

Using SLAM (short for Simultaneous Localization and Mapping) techniques, you will be able to execute autonomous navigation with GoPiGo3.

SLAM is a technique used in robotics to explore and map an unknown environment while estimating the pose of the robot itself. As it moves all around, it will be acquiring structured information of the surroundings by processing the raw data coming from its sensors.

For optimal and easy-to-understand coverage of the topic of SLAM, we will implement a 360º-coverage Laser Distance Sensor (LDS) in the virtual robot. 

There are low-cost versions of this sensor technology, such as EAI YDLIDAR X4 (available at https://www.robotshop.com/es/es/escaner-laser-360-ydlidar-x4.html), which is the one we will make use of in the next chapter.


### Install ROS navigation & SLAM packages
First, let's prepare your machine with the required ROS packages needed for the navigation stack (http://wiki.ros.org/navigation):

In [ ]:
sudo apt install ros-melodic-navigation

And finally the slam_gmapping package, that is already available in its binary version (https://wiki.ros.org/slam_gmapping)

In [ ]:
sudo apt-get install ros-melodic-slam-gmapping

Open the .bashrc file and verify the ROS_IP and ROS_MASTER_URI environment variables and source to the proper workspace:

source ~/rUBotCoop_LabProject/devel/setup.bash

export ROS_IP=192.168.18.83

export ROS_MASTER_URI=http://192.168.18.83:11311

### Navigation MAP generation
SLAM allows the robot to build a map of the environment using the following two sources of information: 
- Robot pose estimation, coming from the internal odometry (rotary encoders) and IMU sensor data 
- Distance to objects, obstacles and walls, coming from distance sensors, the LDS in particular 

In its most basic version, a map includes two-dimensional information, 

The following diagram shows the map generated using SLAM in ROS

In such a two-dimensional map, the free areas and occupied areas are drawn in different intensities of gray in 8-bit format (0-255 range).
A value of -1 is assigned to unknown areas. 

Map information is stored using two files: 
- A .pgm format file, known as portable graymap format. 
- A .yaml file containing the configuration of the map. 
See the following example of its content:

In [ ]:
image: ./test_map.pgm
resolution: 0.010000
origin: [-20.000000, -20.000000, 0.000000]
negate: 0
occupied_thresh: 0.65
free_thresh: 0.196

The most interesting parameters are the last two:  
- occupied_thresh = 0.65 means that a cell is considered as occupied if its probability is above 65%. 
- free_thresh = 0.196 establishes the threshold value below which the cell is considered free, that is, 19.6%. 
Given the size in pixels of the image, it is straightforward to infer the physical dimension of the cells in the map. This value is indicated by the resolution parameter, that is, 0.01 meter/pixel.

Building the map using a Gazebo simulation involves employing the following workflow: 
- Launch the robot model within a modeled environment. 
- Launch the mapping ROS package. 
- Launch a special visualization in RViz that lets us see the areas the robot is scanning as it moves. 
- Teleoperate the robot to make it cover as much as possible of the surface of the virtual environment. 
- Once the exploration is finished, save the map, generating the two files in the formats indicated in the preceding section, that is, .pgm and .yaml.

## Navigation & SLAM with gopigo3

Once your robot has generated a map, it will use it to plan a path to a given target destination. The process of executing such a plan is called navigation, and involves the following steps: 
- Launch the robot model within the modeled environment. This step is the same as the first step in the SLAM process described earlier. 
- Provide the costmap that the robot built before. Bear in mind that the map is a characteristic of the environment, not of the robot. Hence, you can build the map with one robot and use the same map in navigation for any other robot you put in the same environment.
- Set up the navigation algorithm. We will use the Adaptive Monte Carlo Localization (AMCL) algorithm, the most common choice for effective navigation.
- Launch a RViz visualization that will let you visualize the robot in the environment and easily mark the target pose (position and orientation) that it should achieve. 
- Let the robot navigate autonomously to the target location. At this point, you can relax and enjoy watching how the GoPiGo3 drives to the indicated position while avoiding the obstacles and minimizing the distance it has to cover. 
Should you want the robot to navigate to another location, you just have to indicate it in RViz once it has reached the previous target.


Let's follow these steps to build the map of a simple Gazebo world called stage_2.world:

1. Launch the robot model within a modeled environment by running the following line of code:

In [ ]:
roslaunch virtual_slam gopigo3_world.launch world:=stage_2.world

2. Launch the SLAM mapping ROS package, including an RViz visualization that superimposes the virtual model of the robot with the actual scan data:

In [ ]:
roslaunch virtual_slam gopigo3_slam.launch

3. Teleoperate the robot to make it cover as much as possible of the surface of the current Gazebo world. Let's do this as usual with the teleoperation package:

In [ ]:
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel

As you move the robot, the LDS sensor will acquire scan data from the unknown areas, and you will receive feedback in the RViz window.

4. Once you've finished the exploration, save the map, generating two files of the formats indicated in the preceding SLAM process subsection, that is, .pgm and .yaml

In [ ]:
rosrun map_server map_saver -f ~/rUBotCoop_LabProject/map_stage_2

You will get two files in the root folder of your workspace: map_stage_2.pgm and map_stage_2.yaml.

Provided with the map, we are ready to perform robot navigation with the GoPiGo3.

<img src="./Images/03_slam_map1.png">

## Driving along a planned trajectory using navigation

First, close all Terminals. Then, as in the SLAM process, let's proceed step by step to perform some navigation: 
Kill the previous Gazebo process:
killall gzserver && killall gzclient

1. Launch the robot model within the modeled environment. This step is the same as the first step in the SLAM process:

In [ ]:
roslaunch virtual_slam gopigo3_world.launch world:=stage_2.world

2. Set up the navigation algorithm and launch RViz. We will use AMCL, the most common choice for effective navigation.

In this step, we also provide the costmap that the robot built before. To do this, you just have to reference the .yaml map file you created before. Make sure that the corresponding .pgm file has the same name and is placed in the same location.

In [ ]:
roslaunch virtual_slam gopigo3_navigation.launch map_file:=$HOME/rUBotCoop_LabProject/map_stage_2.yaml

3. The RViz window, shown in the following screenshot, lets you visualize the robot in the environment and mark the target pose (position and orientation) that it should achieve:
First of all, you have to tell the robot that this is the initial pose by pressing the 2D Pose Estimate button. Then, mark it on screen (in this particular case, it isn't necessary, since the initial pose is the same as the one the robot had when it started to build the map)

Afterward, you can press 2D Nav Goal button and set the target to the bottom-left corner by clicking the left mouse button. Release the mouse when the arrow has the desired orientation. After releasing, the robot will compute the path to follow and start navigating autonomously.

The orientation of the red arrow tells the GoPiGo3 in what direction it should stay facing once it has arrived at the target, and the curved line going from the robot to the target is the planned path. Since it has a map of the environment available, the robot is able to plan a path that avoids the obstacles.

The blue square around the robot represents the local window for obstacle avoidance planning. This is used by the Dynamic Window Approach (DWA) method, which generates a local path that efficiently evades the obstacles. The DWA method performs the calculations taking into account the robot's dynamics, in particular, its limited velocity and acceleration.

<img src="./Images/03_slam_map2.png">